In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from nltk.stem.porter import PorterStemmer
import nltk
import re, string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer,WordNetLemmatizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve
#from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import pickle


In [31]:
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Load the SpaCy language model
nlp = spacy.load("en_core_web_sm")

# Initialize stop words and lemmatizer
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tong\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tong\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [33]:
def preprocess_text(text):
    # 1. Convert to lowercase
    text = text.lower()
    # 2. Remove punctuation and numbers
    text = ''.join([char if char.isalpha() or char.isspace() else ' ' for char in text])
    # 3. Tokenization
    doc = nlp(text)
    # 4. Lemmatization and removal of stop words
    tokens = [lemmatizer.lemmatize(token.text) for token in doc if token.text not in stop_words]
    
    return ' '.join(tokens)


### Train the Word2Vec model using my own data ,which can make the vector i got more suit for my task


In [35]:

# Load the preprocessed data from the CSV file
data = pd.read_csv("final_preprocessed_data.csv")

# Ensure all values in 'Message' are strings
data['Message'] = data['Message'].fillna('')  # Fill NaN with empty strings
data['Message'] = data['Message'].astype(str)  # Convert all to string

# Filter out 20,000 samples with label 0 and 20,000 samples with label 1
label_0_data = data[data['label'] == 0].head(20000)  # Filter for label = 0
label_1_data = data[data['label'] == 1].head(20000)  # Filter for label = 1

# Concatenate the two datasets to create the final training set
train_data = pd.concat([label_0_data, label_1_data])

# Print the shape of the final training data to ensure correct size
print(f"Final training data shape: {train_data.shape}")

# Extract the 'Message' column (assuming it's the text data column) and split each message into words
sentences = train_data['Message'].apply(lambda x: x.split()).tolist()  # Split each sentence into a list of words

# Import Word2Vec from Gensim for training the word embeddings
from gensim.models import Word2Vec

# Train the Word2Vec model using the processed sentences
model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)

# Save the trained Word2Vec model for later use
model.save("word2vec_final.model")

# The model can be loaded later using the following line:

# model = Word2Vec.load"word2vec_final.model")


Final training data shape: (40000, 2)


# train my own word2vec model

### Verify that my word2Vec model is valid


In [36]:
# Load the trained Word2Vec model
word2vec_model = Word2Vec.load("word2vec_final.model")

# Get the word vector for a specific word (e.g., 'hello')
vector = word2vec_model.wv['hi']

# Get the top 5 most similar words to 'hello'
similar_words = word2vec_model.wv.most_similar('hi', topn=5)
print(similar_words)


[('hello', 0.9813039898872375), ('hunjan', 0.9581018686294556), ('kolkata', 0.9488511085510254), ('rfa', 0.9410589337348938), ('friendly', 0.9337308406829834)]


## Calculate NBLCR term weight


In [58]:
# Load the preprocessed data from the CSV file
data = pd.read_csv("data_preprocessed_val.csv")
# Ensure all values in 'Message' are strings
data['text'] = data['text'].fillna('')  # Fill NaN with empty strings
data['text'] = data['text'].astype(str)  # Convert all to string


In [59]:
from collections import defaultdict

def compute_class_word_frequencies(data, labels):
    class_0_words = Counter()
    class_1_words = Counter()
    total_class_0_words = 0
    total_class_1_words = 0
    
    for message, label in zip(data, labels):
        words = message.split()  # Split the message into words
        if label == 0:
            class_0_words.update(words)
            total_class_0_words += len(words)
        else:
            class_1_words.update(words)
            total_class_1_words += len(words)
    
    return class_0_words, class_1_words, total_class_0_words, total_class_1_words

# Now we can compute NBLCR weights
def compute_nblcr_weights(class_0_words, class_1_words, total_class_0_words, total_class_1_words):
    nblcr_weights = {}
    
    all_words = set(class_0_words.keys()).union(set(class_1_words.keys()))
    
    for word in all_words:
        p_w_class_0 = class_0_words.get(word, 0) / total_class_0_words
        p_w_class_1 = class_1_words.get(word, 0) / total_class_1_words
        
        # NBLCR weight: log ratio of probabilities
        if p_w_class_1 > 0 and p_w_class_0 > 0:
            nblcr_weight = np.log(p_w_class_1 / p_w_class_0)
        else:
            nblcr_weight = 0
        
        nblcr_weights[word] = nblcr_weight
    
    return nblcr_weights




In [60]:
data

,text,label
0,someone buggered stupid redirections player ...,1
1,well known many people many wikipedia account ...,1
2,ignorant piece sht,1
3,want masturbate,1
4,rt natashaallen sexist male comedian much funn...,1
...,...,...
25087,simpson cast member please stop removing ed...,0
25088,p case mistaken asked second opinion wikipedia...,0
25089,thought pull back mkr mkr,0
25090,let conduct talk talkcampbells soup can editor...,0


In [61]:
# Extract messages and labels
messages = data['text']
labels = data['label']

# Compute word frequencies
class_0_words, class_1_words, total_class_0_words, total_class_1_words = compute_class_word_frequencies(messages, labels)

# Compute NBLCR weights for all words
nblcr_weights = compute_nblcr_weights(class_0_words, class_1_words, total_class_0_words, total_class_1_words)

## Generating Sentence Vector Based on NBLCR

In [62]:
def compute_sentence_vector_nblcr(sentence, word2vec_model, nblcr_weights):
    words = sentence.split()  # Split sentence into words
    sentence_vector = np.zeros(word2vec_model.vector_size)  # Initialize sentence vector with zeros
    total_weight = 0.0  # Initialize weight accumulator
    
    for word in words:
        if word in word2vec_model.wv:  # Check if the word exists in the Word2Vec model
            weight = nblcr_weights.get(word, 0)  # Get NBLCR weight for the word (default is 0)
            sentence_vector += weight * word2vec_model.wv[word]  # Add weighted word vector to sentence vector
            total_weight += abs(weight)  # Accumulate weight
    
    # Normalize the sentence vector by the total weight if there are any weights
    if total_weight > 0:
        sentence_vector /= total_weight
    
    return sentence_vector



In [63]:
# Now, we can compute sentence vectors for all messages
sentence_vectors = np.array([
    compute_sentence_vector_nblcr(text, word2vec_model, nblcr_weights) 
    for text in data['text']
])


In [64]:
import pickle

#Save the merged nblcr_weights
with open("nblcr_weights_combined.pkl", "wb") as f:
    pickle.dump(nblcr_weights, f)
    print("Combined NBLCR weights have been saved to 'nblcr_weights_combined.pkl'.")


Combined NBLCR weights have been saved to 'nblcr_weights_combined.pkl'.


### The resulting Bayesian weighted average sentence vector is used as input


In [66]:
sentence_vectors.shape

(25092, 100)

### Building and training models



In [67]:
from keras.optimizers import Adam
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Get the labels
y = data['label']  # Assuming 'label' column contains the labels

# Initialize KFold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation

# Define the neural network structure using Functional API
input_layer = Input(shape=(100,))  # Assuming 100 features after processing
x = Dense(256, activation='relu')(input_layer)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(2, activation='softmax')(x)  # 2 classes for binary classification

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(),  # Use Adam optimizer
              loss='sparse_categorical_crossentropy',  # Sparse categorical crossentropy for integer labels
              metrics=['accuracy'])


In [68]:
# Print model structure
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │        25,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 59,010 (230.51 KB)

 Trainable params: 59,010 (230.51 KB)

 Non-trainable params: 0 (0.00 B)

In [69]:

# Function to evaluate model using KFold
def evaluate_model(model, X, y, kf):
    accuracy_scores = []
    classification_reports = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Fit the model
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

        # Predict the classes
        predictions = np.argmax(model.predict(X_test), axis=1)

        # Collect the metrics
        accuracy_scores.append(accuracy_score(y_test, predictions))
        classification_reports.append(classification_report(y_test, predictions))

    avg_accuracy = np.mean(accuracy_scores)
    avg_classification_report = classification_reports

    return avg_accuracy, avg_classification_report

# Evaluate the model using KFold cross-validation
accuracy, report = evaluate_model(model, sentence_vectors, y, kf)

# Print results
print(f"Average Accuracy: {accuracy}")
print("Classification Report (from each fold):")
for idx, fold_report in enumerate(report):
    print(f"\nFold {idx+1} Classification Report:")
    print(fold_report)


157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step
Average Accuracy: 0.8563289710340245
Classification Report (from each fold):

Fold 1 Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      2674
           1       0.84      0.84      0.84      2345

    accuracy                           0.85      5019
   macro avg       0.85      0.85      0.85      5019
weighted avg       0.85      0.85      0.85      5019


Fold 2 Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.85      0.86      2698
           1       0.83      0.86      0.84      2321

    accuracy                           0.85      5019
   macro avg       0.85      0.85      0.85      5019
weighted avg       0.85      0.85      0.85    

In [70]:
# Save the model
model.save('trained_model.h5')  # Save the model as a .h5 file
print("Model saved as 'trained_model.h5'.")


Model saved as 'trained_model.h5'.
